In [1]:
import tensorflow as tf

#显示用cpu or gpu
#tf.debugging.set_log_device_placement(True)

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2228 - accuracy: 0.9344
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0966 - accuracy: 0.9699
Epoch 3/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0682 - accuracy: 0.9788
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0549 - accuracy: 0.9823
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0764 - accuracy: 0.9781


[0.07635503262281418, 0.9781000018119812]

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(
    nrows=2,
    ncols=5,
    sharex=True,
    sharey=True, )

ax = ax.flatten()
for i in range(10):
    img = x_test[y_test == i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.show()

In [ ]:
y_test[0:10]

# 2、房产测试 神经网络+随机参数搜索

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import matplotlib as mpl
import pandas as pd
import numpy as np
import sklearn
import time
import sys
import os

# 加载数据
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing.data.shape)   # (20640, 8)
print(housing.target.shape) # (20640, )

# 切分数据集
from sklearn.model_selection import train_test_split
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape) # (11610, 8) (11610,)
print(x_valid.shape, y_valid.shape) # (3870, 8) (3870,)
print(x_test.shape, y_test.shape)   # (5160, 8) (5160,)

# 数据归一化 x = (x - u) / d
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

# 建模
# RandomizedSearchCV
# 1. 转化为sklearn的model
# 2. 定义参数集合
# 3. 搜索参数

def build_model(hidden_layers = 1,layer_size = 30,learning_rate = 3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(layer_size, activation='relu',input_shape=x_train.shape[1:]))
    
    for _ in range(hidden_layers - 1):
        model.add(keras.layers.Dense(layer_size,activation = 'relu'))    
    model.add(keras.layers.Dense(1))
    
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss = 'mse', optimizer = optimizer)
    return model


sklearn_model = keras.wrappers.scikit_learn.KerasRegressor(build_fn = build_model)
#sklearn_model.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=5, min_delta=1e-2)]
# history = sklearn_model.fit(x_train_scaled, y_train,
#                             epochs = 10,
#                             validation_data = (x_valid_scaled, y_valid),
#                             callbacks = callbacks)

# 超参数搜索
from scipy.stats import reciprocal
# f(x) = 1/(x*log(b/a)) a <= x <= b
param_distribution = {
    "hidden_layers":[1, 2, 3, 4],
    "layer_size": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),}

from sklearn.model_selection import RandomizedSearchCV
random_search_cv = RandomizedSearchCV(sklearn_model,
                                      param_distribution,
                                      n_iter = 10,
                                      cv = 3,
                                      n_jobs = 1)
random_search_cv.fit(x_train_scaled, y_train, epochs = 100,
                     validation_data = (x_valid_scaled, y_valid),
                     callbacks = callbacks)
# cross_validation: 训练集分成n份，n-1训练，最后一份验证.cv参数来控制
print('最佳参数：', random_search_cv.best_params_)    # 返回最佳参数组合
print('最佳评分：', random_search_cv.best_score_)     # 返回最佳评分 sklearn中回归问题默认mse
print('最佳模型：',random_search_cv.best_estimator_) # 返回最佳模型

# 得到最佳模型并预测
model = random_search_cv.best_estimator_.model
model.evaluate(x_test_scaled, y_test)

(20640, 8)
(20640,)
(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)
Epoch 1/100
242/242 [==============================] - 0s 2ms/step - loss: 1.0114 - val_loss: 0.8123
Epoch 2/100
242/242 [==============================] - 0s 2ms/step - loss: 1.0530 - val_loss: 0.6071
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5366 - val_loss: 0.5224
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4794 - val_loss: 0.4916
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4573 - val_loss: 0.4745
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4463 - val_loss: 0.4514
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4312 - val_loss: 0.4428
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4231 - val_loss: 0.4307
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4141 - val_loss: 0.4241
Epoch 10/100
24

Epoch 14/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3522 - val_loss: 0.3716
Epoch 15/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3482 - val_loss: 0.3583
Epoch 16/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3430 - val_loss: 0.3619
Epoch 17/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3403 - val_loss: 0.3503
Epoch 18/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3368 - val_loss: 0.3625
Epoch 19/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3343 - val_loss: 0.3460
Epoch 20/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3308 - val_loss: 0.3500
Epoch 21/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3273 - val_loss: 0.3484
Epoch 22/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3249 - val_loss: 0.3395
Epoch 23/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3225

Epoch 46/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4586 - val_loss: 0.4972
Epoch 47/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4565 - val_loss: 0.4948
Epoch 48/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4544 - val_loss: 0.4923
Epoch 49/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4524 - val_loss: 0.4901
Epoch 50/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4505 - val_loss: 0.4878
Epoch 51/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4486 - val_loss: 0.4858
Epoch 52/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4470 - val_loss: 0.4836
Epoch 53/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4451 - val_loss: 0.4819
Epoch 54/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4437 - val_loss: 0.4798
Epoch 55/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4419

242/242 [==============================] - 1s 3ms/step - loss: 0.3425 - val_loss: 0.3701
Epoch 14/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3383 - val_loss: 0.3582
Epoch 15/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3343 - val_loss: 0.4157
Epoch 16/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3335 - val_loss: 0.3497
Epoch 17/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3301 - val_loss: 0.3551
Epoch 18/100
242/242 [==============================] - 0s 2ms/step - loss: 0.3265 - val_loss: 0.3546
Epoch 19/100
242/242 [==============================] - 1s 2ms/step - loss: 0.3240 - val_loss: 0.3461
Epoch 20/100
242/242 [==============================] - 1s 3ms/step - loss: 0.3215 - val_loss: 0.3421
Epoch 21/100
121/121 [==============================] - 0s 1ms/step - loss: 0.3195
Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 1.0458 - val_loss: 0.6390
Epoch 2/100
2

# 3.房价xgb + GridSearchCV

In [ ]:
import xgboost as xgb
from xgboost import plot_importance
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
# 加载数据
boston = load_boston()
x = np.array(boston.data)
y = np.array(boston.target)
x_test = x[-3:,:]
y_test = y[-3:]
x_train, x_valid, y_train, y_valid = train_test_split(x[:-3,:], y[:-3], test_size=0.2)
train = xgb.DMatrix(x_train, label=y_train)
valid = xgb.DMatrix(x_valid, label=y_valid)
test = xgb.DMatrix(x_test, label=y_test)
print('x:',len(x))
print('y:',len(y))
print('x_test:',len(x_test),x_test)
print('y_test:',len(y_test),y_test)
print('x_train, y_train',len(x_train), len(y_train))

In [ ]:
# 参数设置
params = {
    'booster': 'gbtree',
    'objective':'reg:linear',
    'max_depth':6,
    'eta':0.1,
    'silent':1,
    'gamma':0,
    'min_child_weight':1,
    'subsample':1,
    'colsample_bytree':1,
    'colsample_bylevel':1,
    'lambda':1,
    'alpha':0,
    'nthread':-1,
    'eval_metric':'rmse',
    'seed':0
}
# 验证集
evallist  = [(valid,'eval')]
# 模型训练
model = xgb.train(params.items(), train, 100, evals=evallist)
# 特征重要性
plot_importance(model)
plt.show()
# 预测
res = model.predict(test)
print('预测结果：')
print(res)
print(y_test)

# 3.1 参数调优
XGBoost调参主要用到其CV方法以及sklearn中的GridSearchCV、RandomizedSearchCV方法。这里继续以Boston房价为例。

首先，为了提高效率，设置eta为一个较大的值，确定其迭代次数，最后的时候再调小这个参数，进一步提高准确率。

In [ ]:
params = {
    'booster': 'gbtree',
    'objective':'reg:linear',
    'max_depth':6,
    'eta':0.1,
    'silent':1,
    'gamma':0,
    'min_child_weight':1,
    'subsample':1,
    'colsample_bytree':1,
    'colsample_bylevel':1,
    'lambda':1,
    'alpha':0,
    'nthread':-1,
    'eval_metric':'rmse',
    'seed':0
}
cv = xgb.cv(params, train, num_boost_round=10000, nfold=3, metrics='rmse', early_stopping_rounds=10, seed=0)
print('Best round：',cv.shape[0])
print('Best score：',cv.values[-1,:])

从结果可以看出，eta=0.1的条件下，最优迭代次数是：70次，把相应的参数如此设置。

接下来使用GridSearchCV调其他参数，调整其他参数需要使用sklearn下的XGBoost。

In [ ]:
# 网格调参
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# 模型训练
model = xgb.XGBRegressor(max_depth=9, learning_rate=0.1, n_estimators=130, silent=True, 
                         objective='reg:linear', nthread=-1, gamma=0.1, min_child_weight=0, 
                         max_delta_step=0, subsample=0.5, colsample_bytree=0.8, colsample_bylevel=0.4, 
                         reg_alpha=0.4, reg_lambda=0.8, scale_pos_weight=1, base_score=0.5, seed=0, 
                         missing=None)
model.fit(x_train,y_train)
val = model.predict(x_valid)
print(mean_squared_error(y_valid,val))

# 特征重要性
plot_importance(model)
plt.show()

res = model.predict(x_test)
print('预测结果：')
print('res:',res)
print('y_test',y_test)

para = {
     'learning_rate':[0.3,0.1,0.05,0.03,0.01],
     'n_estimators':np.arange(50,200,20),
     'max_depth':np.arange(3,10,1),
     'gamma':np.arange(0.1,1,.1),
#     'min_child_weight':np.arange(0,5,1),
#     'subsample':np.arange(0.3,1,.1),
#     'colsample_bytree':np.arange(0.3,1,.1),
#     'colsample_bylevel':np.arange(0.3,1,.1),
#     'reg_alpha':np.arange(0,0.5,.1),
#     'reg_lambda':np.arange(0.5,1,.1),
}
cv = GridSearchCV(estimator=model,param_grid=para,scoring='neg_mean_squared_error',cv=5,verbose=1,n_jobs=-1)
cv.fit(x_train,y_train)
print("Best params:",cv.best_params_)
print('Best score:',cv.best_score_)